#### CSC 296S Deep Learning (Spring 2026)

#### Dr. Haiquan Chen, Dept of Computer Scicence

#### California State University, Sacramento




## Project 1

In [ ]:
# What version of Python do you have?

import tensorflow as tf
import sys
import sklearn as sk
import pandas as pd

Python 3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 11:23:37) [Clang 14.0.6 ]
Pandas 2.3.2
Scikit-Learn 1.7.1

Tensor Flow Version: 2.20.0
Keras Version: 3.11.3
